In [1]:
from pathlib import Path
import exifread
import sys
import time
import datetime

sys.path.append("..")
from semif_utils.mongo_utils import to_db, Connect
from semif_utils.datasets import ImageMetadata
import shutil

In [9]:
imgs = Path("../data/semifield-developed-images-trial/NC_2022-03-04/images").glob("*.jpg")
imgs = [x for x in imgs]

In [10]:
def get_metadata(path):
    """ Change image filename to [state]_[row_number]_[stop]_[unixtime from image metadata]
    datetime to unix time in milliseconds
    """
    # Open image file for reading (must be in binary mode)
    f = open(path, 'rb')
    # Return Exif tags
    tags = exifread.process_file(f,details=False)
    f.close()
    meta = {}
    for x, y in tags.items():
        newval = y.values[0] if type(y.values) == list and len(y.values) == 1 else y.values
        if type(newval) == exifread.utils.Ratio:
            newval = str(newval)
        meta[x.rsplit(" ")[1]] = newval
        pop_list = ["MakerNote", "UserComment", "ImageDescription", "ApplicationNotes"]
        meta.pop(x.rsplit(" ")[1])  if x.rsplit(" ")[1] in pop_list else None
    return meta



def computer_now_2_unixmilli():
    ms = datetime.datetime.now()
    unixmilli = time.mktime(ms.timetuple()) * 1000
    return unixmilli

def exifdatetime_2_unixmilli(imgmeta):
    dt = imgmeta["DateTime"]
    dt = datetime.datetime.strptime(dt, '%Y:%m:%d %H:%M:%S')
    milliunix = time.mktime(dt.timetuple()) * 1000
    return milliunix

def unixmilli_2_datetime(unixmilli_time):
    unix_time = unixmilli_time / 1000
    date_time = datetime.datetime.fromtimestamp(int(unix_time))
    dt_str = date_time.strftime('%Y-%m-%d %H:%M:%S')
    return dt_str

def folder_date_2_unixmilli(date):
    dtp = date.split("-")
    
    print(dtp)
    
    date_format = datetime.date(int(dtp[0]), int(dtp[1]), int(dtp[2]))

    unixmilli = time.mktime(date_format.timetuple()) * 1000
    return unixmilli

img = imgs[0]
meta = get_metadata(img)
print("Datetime from image exif metadata: ", meta["DateTime"])
unixmil = exifdatetime_2_unixmilli(meta) 
print("Unix millisecond time: ", unixmil)
dt = unixmilli_2_datetime(unixmil)
print("Back to datetime string but in another format: ", dt)
folderdate = folder_date_2_unixmilli("2022-03-29")


Datetime from image exif metadata:  2022:03:04 13:15:31
Unix millisecond time:  1646421331000.0
Back to datetime string but in another format:  2022-03-04 13:15:31
['2022', '03', '29']


In [12]:
imgs = Path("../data/semifield-developed-images-trial/NC_2022-03-04/images").glob("*.jpg")
imgs = [x for x in imgs]
for img in imgs:

    meta = get_metadata(img)
    unixmil = exifdatetime_2_unixmilli(meta)
    fstem = img.stem
    parts = fstem.split("_")
    # state = parts[0]
    state  = "NC"
    stop = parts[-1]
    row = parts[-2].replace("Row", "")
    src = img

    dst = Path(img.parent, f"{state}_{row}_{stop}_{unixmil}.jpg")
    print(dst)
    # shutil.move(src, dst)
    

../data/semifield-developed-images-trial/NC_2022-03-04/images/NC_6_2_1646421331000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-04/images/NC_6_1_1646421311000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-04/images/NC_3_2_1646419293000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-04/images/NC_2_1_1646419092000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-04/images/NC_1_2_1646418772000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-04/images/NC_4_2_1646420037000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-04/images/NC_2_3_1646419132000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-04/images/NC_4_3_1646420347000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-04/images/NC_1_1_1646418752000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-04/images/NC_5_2_1646420631000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-04/images/NC_3_1_1646419274000.0.jpg
../data/semifield-dev

## Change Downscaled_Photos

In [17]:
imgs = Path("../data/semifield-developed-images-trial/NC_2022-03-29/autosfm/downscaled_photos").glob("*.jpg")
imgs = [x for x in imgs]
for img in imgs:
    meta = get_metadata(img)
    unixmil = exifdatetime_2_unixmilli(meta)
    fstem = img.stem
    fstem = fstem.replace("row", "")
    parts = fstem.split("_")
    state = "NC"
    stop = parts[-1]
    row = parts[-2].replace("row", "")
    src = img
    dst = Path(img.parent, f"{state}_{row}_{stop}_{unixmil}.jpg")
    print(dst)

    # shutil.move(src, dst)

../data/semifield-developed-images-trial/NC_2022-03-29/autosfm/downscaled_photos/NC_1_3_1648579026000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-29/autosfm/downscaled_photos/NC_3_4_1648579561000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-29/autosfm/downscaled_photos/NC_4_4_1648579313000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-29/autosfm/downscaled_photos/NC_5_6_1648580258000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-29/autosfm/downscaled_photos/NC_6_6_1648580423000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-29/autosfm/downscaled_photos/NC_6_4_1648580383000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-29/autosfm/downscaled_photos/NC_6_2_1648580342000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-29/autosfm/downscaled_photos/NC_4_3_1648579293000.0.jpg
../data/semifield-developed-images-trial/NC_2022-03-29/autosfm/downscaled_photos/NC_2_4_1648579783000.0.jpg
../data/semifield-developed-

In [85]:
imgs = Path("../blob_container/semifield-developed-images-trial/NC_2022-04-05").glob("*.jpg")
imgs = [x for x in imgs]
for img in imgs:
    fstem = img.stem
    parts = fstem.split("_")[1:]
    join_parts = "_".join(parts)
    dst = Path(img.parent, f"NC_{join_parts}.jpg")
    src = img
    # shutil.move(src, dst)

In [22]:
# change folder name
dir = Path("../blob_container/semifield-developed-images-trial/NC_2022-04-05")
dirdate = dir.name.split("_")[1]
dir_unixmilli = folder_date_2_unixmilli(dirdate)
print(dir_unixmilli)
dt = unixmilli_2_datetime(dir_unixmilli)
print(dt)
# Path(f"../blob_container/semifield-developed-images-trial/NC_{dir_unixmilli}").mkdir()



['2022', '03', '29']
1648530000000.0
2022-03-29 00:00:00
